# 数据处理

## 数据处理 - 自来水


>     col 0	5位数字ID，前两位代表“县”
>     col 1	多数情况代表地点，少数情况代表企业
>     col 5	水量
>     col 9	硝酸盐浓度
>     col 16	Na浓度
>     col 18	Cl浓度
>     col 19	硬度
>     col 20	蒸发残留物
>     col 22	TOC
> 
> 以上数据，需要对同一个ID（col 0），以水量（col 5）为权重，加权平均  
> 然后对于同一个县（col 0的前两位）所有的ID，以该地点（col 1）的人口为权重，加权平均


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)   # 每行最大字符
pd.set_option('precision', 3)         # 显示数字精度

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['axes.unicode_minus'] = False     # 显示数字负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 显示中文字体
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
plt.style.use('seaborn-whitegrid')

import seaborn as sns


### 预览

In [25]:
water_df = pd.read_excel("../water_map_jp 180917.xlsx", sheet_name="raw")

In [26]:
print('预览前几行')
water_df.head(5)

预览前几行


,事業主体ID,事業主体名,浄水場名,水源名,原水の種類,1日平均浄水量,Unnamed: 6,大腸菌(定量)(MPN/100ml),大腸菌(定性),硝酸態窒素及び亜硝酸態窒素,...,銅及びその化合物,ナトリウム及びその化合物,マンガン及びその化合物,塩化物イオン,カルシウム、マグネシウム等(硬度),蒸発残留物,陰イオン界面活性剤,有機物(TOCの量),色度,濁度
0,01-005,北海道稚内市,01-00萩ヶ丘浄水場,声問川水系北辰ダム,ダム直接,18859,10,0,0,0.50,...,<0.01,11.3,0.016,14.5,16,70,<0.02,1.5,10.3,1.7
1,01-005,北海道稚内市,02-00沼川第1浄水場,声問川水系北辰ダム,ダム直接,221,10,0,0,0.50,...,<0.01,11.3,0.016,14.5,16,70,<0.02,1.5,10.3,1.7
2,01-005,北海道稚内市,03-00沼川第2浄水場,声問川水系北辰ダム,ダム直接,810,10,0,0,0.50,...,<0.01,11.3,0.016,14.5,16,70,<0.02,1.5,10.3,1.7
3,01-010,北海道浦河町,01-00野深浄水場,1号井戸、2号井戸,浅井戸水,3800,1,0,0,0.36,...,0,3.7,<0.001,3.5,29,60,<0.02,0.3,<1.0,<0.1
4,01-015,北海道木古内町,01-00木古内浄水場,木古内川水系支流中野川,伏流水,2010,13,0,0,0.17,...,0.02,10,<0.001,12.7,21,70,<0.02,0.5,2,0.3


In [92]:
def explore_df(df):
  print('==== row, col', df.shape)
#   print(df.columns)
#   print('==== head 3')
#   print(df.head(3))
#   print('==== tail 3')
#   print(df.tail(3))
  print('==== describe ====')
  print(df.describe())
  print('==== info ====')
  print(df.info())
  
# explore_df(df)

def find_in_df(df, col, pat):
  ''' 在 col 中 查找命中 pat 的数据
      如果 pat 是 字符串, 视为模糊查找 
      如果 pat 是 list, 或者非字符串型, 视为精确查找 '''
  if isinstance(pat, str):
    return df[df[col].str.contains(pat)]
  if not isinstance(pat, (list, set)):
    pat = [pat]
  return df[df[col].isin(pat)]


In [27]:
print('行, 列', water_df.shape)
print(water_df.columns)

行, 列 (3619, 25)
Index(['事業主体ID', '事業主体名', '浄水場名', '水源名', '原水の種類', '1日平均浄水量', 'Unnamed: 6', '大腸菌(定量)(MPN/100ml)', '大腸菌(定性)', '硝酸態窒素及び亜硝酸態窒素', 'フッ素及びその化合物', 'ホウ素及びその化合物', '四塩化炭素', 'アルミニウム及びその化合物', '鉄及びその化合物',
       '銅及びその化合物', 'ナトリウム及びその化合物', 'マンガン及びその化合物', '塩化物イオン', 'カルシウム、マグネシウム等(硬度)', '蒸発残留物', '陰イオン界面活性剤', '有機物(TOCの量)', '色度', '濁度'],
      dtype='object')


### 保留有用的column, 重命名column

In [28]:
column_map = [
  ('事業主体ID',                       '5位数字ID'),
  ('事業主体名',                       '地点或企业'),
  ('1日平均浄水量',                    '水量'),
  ('硝酸態窒素及び亜硝酸態窒素',        '硝酸盐浓度'),
  ('ナトリウム及びその化合物',          'Na浓度'),
  ('塩化物イオン',                     'Cl浓度'),
  ('カルシウム、マグネシウム等(硬度)',   '硬度'),
  ('蒸発残留物',                       '蒸发残留物'),
  ('有機物(TOCの量)',                  'TOC'),
]

water_df = water_df[[line[0] for line in column_map]]
water_df.columns = [line[1] for line in column_map]
water_df.head(5)

,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
0,01-005,北海道稚内市,18859,0.50,11.3,14.5,16,70,1.5
1,01-005,北海道稚内市,221,0.50,11.3,14.5,16,70,1.5
2,01-005,北海道稚内市,810,0.50,11.3,14.5,16,70,1.5
3,01-010,北海道浦河町,3800,0.36,3.7,3.5,29,60,0.3
4,01-015,北海道木古内町,2010,0.17,10,12.7,21,70,0.5


In [29]:
water_df.describe()
water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3619 entries, 0 to 3618
Data columns (total 9 columns):
5位数字ID    3619 non-null object
地点或企业     3619 non-null object
水量        3591 non-null object
硝酸盐浓度     3619 non-null float64
Na浓度      3619 non-null object
Cl浓度      3619 non-null object
硬度        3619 non-null object
蒸发残留物     3619 non-null object
TOC       3619 non-null float64
dtypes: float64(2), object(7)
memory usage: 254.5+ KB


### 异常值处理

有217行测量结果不是数值型

In [30]:
detect_columns = ['水量', '硝酸盐浓度', 'Na浓度', 'Cl浓度', '硬度', '蒸发残留物', 'TOC',]
df_for_drop = water_df[~water_df[detect_columns].applymap(np.isreal).all(1)]
print(df_for_drop.shape)
df_for_drop.head(10)

(217, 9)


,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
21,01-044,北海道足寄町,2100,0.20,<4.0,1.3,9,80,0.7
81,02-010,青森県深浦町,19,0.04,54,42,<1,230,0.3
109,02-052,青森県六ヶ所村,41,0.20,15.8,9.7,<34,153,0.4
131,02-171,青森県むつ市,(m3)休止中,0.35,15.6,28.2,44,141,0.3
211,03-012,岩手県陸前高田市,"2,908 (m3)休止中",0.85,11.8,18.1,79,139,0.6
212,03-012,岩手県陸前高田市,181(m3)休止中,0.70,6.6,5.6,63,93,0.5
215,03-014,岩手県紫波町,247,4.33,8.7,<10.3,78,<162,0.4
257,04-013,宮城県多賀城市,216(m3)休止中,0.05,178,32.6,10,471,1.3
274,04-074,宮城県加美町,3608,0.02,<10.6,<4.2,<28,142,0.3
275,04-074,宮城県加美町,397,0.02,<9.7,4.4,<24,<137,0.3


In [9]:
# df[df.地点或企业.isin(['北海道足寄町'])]
# df[df.地点或企业.isin(['三重県四日市市'])]
# df_for_drop.index

In [31]:
# 将 "<10.3" 的数值, 替换为 10.3 数值

def replace_lt(x):
  if isinstance(x, str) and x[0] == '<':
    return float(x[1:])
  elif isinstance(x, (float, int)):
    return x
  else:
    raise ValueError(f'cannot parse {x} {type(x)}')


water_df['硝酸盐浓度'] = water_df['硝酸盐浓度'].map(replace_lt)
water_df['Na浓度'] = water_df['Na浓度'].map(replace_lt)
water_df['Cl浓度'] = water_df['Cl浓度'].map(replace_lt)
water_df['硬度'] = water_df['硬度'].map(replace_lt)
water_df['蒸发残留物'] = water_df['蒸发残留物'].map(replace_lt)
water_df['TOC'] = water_df['TOC'].map(replace_lt)


# 将 "2,908 (m3)休止中" 的数值, 替换为 2908 数值

def replace_volume(x):
  if isinstance(x, str) and x.endswith('(m3)休止中'):
    x = x.split('(m3)')[0].strip()
    return int(x.replace(',', '')) if x else np.NaN
  elif isinstance(x, (float, int)):
    return x
  else:
    raise ValueError(f'cannot parse {x} {type(x)}')

water_df['水量'] = water_df['水量'].map(replace_volume)

然后又有47行没有水量, 

一部分来自原表空缺, 

一部分由于原表写的是 '(m3)休止中', 刚才也被替换了 NaN

In [32]:
print(water_df[water_df.isnull().any(1)].shape)
water_df[water_df.isnull().any(1)]


(47, 9)


,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
131,02-171,青森県むつ市,NaN,0.35,15.6,28.2,44.0,141.0,0.3
364,06-041,山形県金山町,NaN,0.20,7.2,9.3,18.0,53.0,0.6
378,07-010,福島県須賀川市,NaN,0.05,10.5,2.2,26.0,139.0,0.3
431,07-067,福島県西郷村,NaN,0.92,7.6,4.9,95.0,170.0,0.5
681,10-001,群馬県高崎市,NaN,0.10,9.8,1.2,38.0,120.0,0.3
688,10-001,群馬県高崎市,NaN,0.91,8.7,2.2,56.0,150.0,0.3
800,10-053,群馬県渋川市,NaN,3.70,8.0,5.0,56.0,142.0,0.3
888,11-038,埼玉県上尾市,NaN,0.05,8.8,5.2,77.0,148.0,0.9
973,12-004,千葉県松戸市,NaN,1.89,16.0,24.1,69.0,160.0,0.8
1190,15-074,新潟県上越市,NaN,0.10,43.0,28.0,39.0,190.0,0.5


In [33]:
# 删去没有水量的数据
water_df.drop(index=water_df[water_df.isnull().any(1)].index, inplace=True)

In [34]:
# 这时所有的行都是有效数字了, 不再出现异常的行
detect_columns = ['水量', '硝酸盐浓度', 'Na浓度', 'Cl浓度', '硬度', '蒸发残留物', 'TOC',]
water_df[~water_df[detect_columns].applymap(np.isreal).all(1)]

,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC


In [35]:
# 总共 3572 行
water_df.shape

(3572, 9)

### 对硝酸盐浓度, Na浓度等, 以水量为权重做加权平均

In [15]:
# test

# df = (pd.DataFrame.from_dict(dict([('ID', [1, 1, 1, 2, 2]),
#                                   ('HeightA', [1, 2, 3, 4, 5]), 
#                                   ('WeightA', [10, 10, 10, 0, 3]),
#                                   ('HeightB', [2, 4, 6, 8, 10]), 
#                                   ('WeightB', [1, 1, 2, 1, 2])]),
#                             ))
# print (df)

# def givewm(weightcolumn):
#   def f1(x): return np.ma.average(x, weights=df.loc[x.index, weightcolumn])
#   return f1

# df2 = df.groupby('ID').agg({'HeightA': givewm('WeightA'),
#                             'HeightB': givewm('WeightB')
#                            })
# print (df2)

In [81]:
def givewm(weightcolumn):
  def f1(x): return np.ma.average(x, weights=water_df.loc[x.index, weightcolumn])
  return f1

water_df_agg = water_df.groupby('地点或企业').agg({
   '5位数字ID': 'first',
   '硝酸盐浓度': givewm('水量'),
   'Na浓度': givewm('水量'),
   'Cl浓度': givewm('水量'),
   '硬度': givewm('水量'),
   '蒸发残留物': givewm('水量'),
   'TOC': givewm('水量'),
}).reset_index()

water_df_agg

,地点或企业,5位数字ID,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
0,三重県いなべ市,24-010,0.903,5.131,4.089,87.891,120.088,0.313
1,三重県亀山市,24-015,0.926,10.730,11.764,53.530,113.528,0.300
2,三重県伊勢市,24-009,1.418,6.296,5.942,56.860,101.628,0.300
3,三重県伊賀市,24-005,0.886,7.629,8.826,50.603,91.713,0.695
4,三重県四日市市,24-004,1.747,9.423,8.506,62.611,114.655,0.263
5,三重県多気町,24-043,2.181,6.274,6.598,64.169,108.914,0.300
6,三重県尾鷲市,24-014,0.200,3.700,3.700,9.000,18.000,0.300
7,三重県御浜町,24-055,0.754,5.800,6.122,23.699,72.398,0.100
8,三重県明和町,24-048,2.492,8.038,7.781,50.781,100.095,0.300
9,三重県朝日町,24-021,0.966,9.793,7.264,87.244,178.179,0.370


### 补充 Enterprise to location.xlsx

> 传了一个Enterprise to location，这里头可以把企业变成地点。  
> 它一个企业同时给好几个市供水，那就把这好几个地点的化学指标定为昨天算出来的该企业的数据。  
> 地点之间，我用空格做分词符。

这个 "Enterprise to location" 提供了 "自来水企业 -> 市" 的映射, 以及 "自来水企业 -> 净水厂" 的映射, 
都是一对多的映射

昨天的数据是 "市 -> 净水厂" 的一对多映射

这三者是什么关系? 
 
另外, 两份数据都有净水厂编号, 但是编号似乎对不上, 这编号没有意义?


> enterprise to location的一对多映射，跟昨天的市->净水厂的一对多映射，可能是互补关系。编号有意义，应该是primary key，在两张表里是一样的。上表可能包含下表，但是反过来不成立

> enterprise.col(A)等价water.col(A)  
> enterprise.col(C)等价water.col(B)  
> enterprise.col(B+D)等价water.col(B)当中的大部分地名

In [37]:
location_df = pd.read_excel("../Enterprise to location.xlsx")

In [38]:
location_df.head(5)

,事業主体ID,県,事業主体名,市町村,浄水場名,水源名,原水の種類,1日平均浄水量,Unnamed: 8,大腸菌(定量)(MPN/100ml),...,ジェオスミン,2-メチルイソボルネオール,非イオン界面活性剤,フェノール類,有機物(TOCの量),pH値,味,臭気,色度,濁度
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源,新宮新水源(新宮新水源、曽我井水源と混,NaN,(m3)休止中,1,0.0,...,<0.000001,<0.000001,<0.005,<0.0005,0.1,6.3,0,0,<0.5,<0.1
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源,川向水源,浅井戸水,2102,4,0.0,...,<0.000001,<0.000001,<0.005,<0.0005,0.1,6.5,0,0,<0.5,<0.1
2,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,01-00立田配水場,県水受水(尾張西部),浄水受水,5406,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,02-00弥富配水場,県水受水(尾張西部),浄水受水,16122,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,03-00佐屋配水場,県水受水(尾張西部),浄水受水,8537,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Enterprise to location.xlsx 保留企业名地名, 除掉该表中的检测数据

location_df = location_df['事業主体ID	県	事業主体名	市町村	浄水場名'.split('\t')]


explore_df(location_df)
location_df.head(5)

==== row, col (257, 5)
==== describe ====
        事業主体ID    県      事業主体名            市町村        浄水場名
count      257  257        257            257         257
unique      49   22         47             47         256
top     28-097   兵庫  淡路広域水道企業団  南あわじ市 洲本市 淡路市  04-00佐野浄水場
freq        46   52         46             46           2
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 5 columns):
事業主体ID    257 non-null object
県         257 non-null object
事業主体名     257 non-null object
市町村       257 non-null object
浄水場名      257 non-null object
dtypes: object(5)
memory usage: 10.1+ KB
None


,事業主体ID,県,事業主体名,市町村,浄水場名
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源
2,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,01-00立田配水場
3,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,02-00弥富配水場
4,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,03-00佐屋配水場


> Enterprise to location可以把企业变成地点  
> 一个企业同时给好几个市供水，那就把这好几个地点的化学指标定为昨天算出来的该企业的数据  
> 地点之间用空格分词

In [49]:
# 检查 location_df 的 事業主体ID 和 water_df 的 ID

water_df_ids = set(water_df['5位数字ID'])
location_df_ids = set(location_df['事業主体ID'])

print('water_df 超出 location_df 的 ID 数量: ', len(water_df_ids - location_df_ids))
print('location_df 超出 water_df 的 ID 数量: ', len(location_df_ids - water_df_ids), '分别是: ',location_df_ids - water_df_ids)


water_df 超出 location_df 的 ID 数量:  940
location_df 超出 water_df 的 ID 数量:  17 分别是:  {'41-004', '23-034', '25-016', '25-030', '43-029', '12-039', '35-020', '06-038', '08-015', '01-101', '01-142', '08-016', '40-096', '04-065', '47-009', '01-126', '02-046'}


那么除了这 17 个 ID 的企业没有线索, 剩下企业的测量数据, 都能从 water_df 里补充上

In [83]:
# 将 water_df_agg 连接到 location_df 上, 通过企业ID
location_merged_df = pd.merge(location_df, water_df_agg, how='inner', on=None, left_on='事業主体ID', right_on='5位数字ID')

location_merged_df.shape

(212, 13)

257 - 17 = 240

但这里只留下来 212 行, 因为这 17 个 ID 有的重复了许多次

In [84]:
location_merged_df.head(5)

,事業主体ID,県,事業主体名,市町村,浄水場名,地点或企业,5位数字ID,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1
2,25-034,滋賀,愛知郡広域行政組合,愛荘町 東近江市,01-00鯰江浄水場,滋賀県愛知郡広域行政組合,25-034,3.548,10.79,16.076,57.79,126.685,0.3
3,25-034,滋賀,愛知郡広域行政組合,愛荘町 東近江市,02-00中戸浄水場,滋賀県愛知郡広域行政組合,25-034,3.548,10.79,16.076,57.79,126.685,0.3
4,02-051,青森,八戸圏域水道企業団,八戸市 七戸町 六戸町 三戸町 五戸町 おいらせ町 南部町,01-00白山浄水場,青森県八戸圏域水道企業団,02-051,4.530,17.20,20.100,152.00,259.000,0.2


In [85]:
# 按 `市町村` 的空格拆开 location_merged_df

location_merged_df = location_merged_df.join(location_merged_df['市町村'].str.split(' ', expand=True) \
                                                   .stack() \
                                                   .reset_index(level=1, drop=True) \
                                                   .rename('市町村split')
                       )
location_merged_df.head(10)

,事業主体ID,県,事業主体名,市町村,浄水場名,地点或企业,5位数字ID,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,市町村split
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1,たつの市
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1,上郡町
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1,佐用町
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1,たつの市
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1,上郡町
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源,兵庫県播磨高原広域事務組合,28-094,2.300,13.00,35.000,76.00,190.000,0.1,佐用町
2,25-034,滋賀,愛知郡広域行政組合,愛荘町 東近江市,01-00鯰江浄水場,滋賀県愛知郡広域行政組合,25-034,3.548,10.79,16.076,57.79,126.685,0.3,愛荘町
2,25-034,滋賀,愛知郡広域行政組合,愛荘町 東近江市,01-00鯰江浄水場,滋賀県愛知郡広域行政組合,25-034,3.548,10.79,16.076,57.79,126.685,0.3,東近江市
3,25-034,滋賀,愛知郡広域行政組合,愛荘町 東近江市,02-00中戸浄水場,滋賀県愛知郡広域行政組合,25-034,3.548,10.79,16.076,57.79,126.685,0.3,愛荘町
3,25-034,滋賀,愛知郡広域行政組合,愛荘町 東近江市,02-00中戸浄水場,滋賀県愛知郡広域行政組合,25-034,3.548,10.79,16.076,57.79,126.685,0.3,東近江市


In [86]:
# 去掉多余字段
location_merged_df.drop(columns=['市町村', '5位数字ID'], inplace=True)
location_merged_df

,事業主体ID,県,事業主体名,浄水場名,地点或企业,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,市町村split
0,28-094,兵庫,播磨高原広域事務組合,01-00新宮新水源,兵庫県播磨高原広域事務組合,2.300,13.000,35.000,76.000,190.000,0.100,たつの市
0,28-094,兵庫,播磨高原広域事務組合,01-00新宮新水源,兵庫県播磨高原広域事務組合,2.300,13.000,35.000,76.000,190.000,0.100,上郡町
0,28-094,兵庫,播磨高原広域事務組合,01-00新宮新水源,兵庫県播磨高原広域事務組合,2.300,13.000,35.000,76.000,190.000,0.100,佐用町
1,28-094,兵庫,播磨高原広域事務組合,03-00川向水源,兵庫県播磨高原広域事務組合,2.300,13.000,35.000,76.000,190.000,0.100,たつの市
1,28-094,兵庫,播磨高原広域事務組合,03-00川向水源,兵庫県播磨高原広域事務組合,2.300,13.000,35.000,76.000,190.000,0.100,上郡町
1,28-094,兵庫,播磨高原広域事務組合,03-00川向水源,兵庫県播磨高原広域事務組合,2.300,13.000,35.000,76.000,190.000,0.100,佐用町
2,25-034,滋賀,愛知郡広域行政組合,01-00鯰江浄水場,滋賀県愛知郡広域行政組合,3.548,10.790,16.076,57.790,126.685,0.300,愛荘町
2,25-034,滋賀,愛知郡広域行政組合,01-00鯰江浄水場,滋賀県愛知郡広域行政組合,3.548,10.790,16.076,57.790,126.685,0.300,東近江市
3,25-034,滋賀,愛知郡広域行政組合,02-00中戸浄水場,滋賀県愛知郡広域行政組合,3.548,10.790,16.076,57.790,126.685,0.300,愛荘町
3,25-034,滋賀,愛知郡広域行政組合,02-00中戸浄水場,滋賀県愛知郡広域行政組合,3.548,10.790,16.076,57.790,126.685,0.300,東近江市


In [99]:
# 随便选一个市, 这里的数据来自 location_merged_df + water_df_agg, 以 企业ID 连接
find_in_df(location_merged_df, col='市町村split', pat='山武市')

,事業主体ID,県,事業主体名,浄水場名,地点或企业,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,市町村split
210,12-071,千葉,山武郡市広域水道企業団,05-00大網配水場,千葉県山武郡市広域水道企業団,3.035,28.484,46.588,89.526,232.366,1.2,山武市
211,12-071,千葉,山武郡市広域水道企業団,06-00松尾配水場,千葉県山武郡市広域水道企業団,3.035,28.484,46.588,89.526,232.366,1.2,山武市


In [100]:
# 同样这个市, 这里的数据来自 water_df_agg 中每个市的水量加权平均
find_in_df(water_df_agg, col='地点或企业', pat='山武市')

# ???

,地点或企业,5位数字ID,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
138,千葉県山武市,12-073,0.17,6.4,5.9,54.0,122.0,0.3
